# Content-based Filtering

Aproach:
- Table with features erstellen, ratings in zweitem table

- sklearn-kNN mit minkowski-distance auf features anwenden, erhalte matrix mit allen Nachbarn eines jeden Films inklusive Abstand

- Wähle für jeden Nutzer und Film die x-nächsten Nachbarn aus, die der Nutzer bewertet hat

- Funktion schreiben, die auf basis des ratings der neighbours das rating eines Filmes vorhersagt.

In [177]:
import pandas as pd
import numpy as np
import string
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.decomposition import PCA
kwargs = dict(random_state=42)

In [178]:
movies = pd.read_csv('../../data/preprocessed/movies_id_updated.csv')
average_ratings = pd.read_csv('../../data/preprocessed/average_ratings.csv')
omdb = pd.read_csv('../../data/preprocessed/omdb_cleaned.csv')
ratings = pd.read_csv('../../data/preprocessed/ratings_clean_std_0.csv')

## Feature Preprocessing

In [179]:
movies.head()

,id,title,imdbID,spanishTitle,imdbPictureURL,year,rtID,rtPictureURL
0,1,Toy story,tt0114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995.0,toy_story,http://content7.flixster.com/movie/10/93/63/10...
1,2,Jumanji,tt0113497,Jumanji,http://ia.media-imdb.com/images/M/MV5BMzM5NjE1...,1995.0,1068044-jumanji,http://content8.flixster.com/movie/56/79/73/56...
2,3,Grumpy Old Men,tt0107050,Dos viejos gruñones,http://ia.media-imdb.com/images/M/MV5BMTI5MTgy...,1993.0,grumpy_old_men,http://content6.flixster.com/movie/25/60/25602...
3,4,Waiting to Exhale,tt0114885,Esperando un respiro,http://ia.media-imdb.com/images/M/MV5BMTczMTMy...,1995.0,waiting_to_exhale,http://content9.flixster.com/movie/10/94/17/10...
4,5,Father of the Bride Part II,tt0113041,Vuelve el padre de la novia (Ahora también abu...,http://ia.media-imdb.com/images/M/MV5BMTg1NDc2...,1995.0,father_of_the_bride_part_ii,http://content8.flixster.com/movie/25/54/25542...


In [180]:
ratings

,Unnamed: 0,user_id,imdbID,rating
0,0,1264,tt0047034,3.5
1,1,213,tt0304141,2.5
2,2,593,tt0369436,3.0
3,3,609,tt1077258,4.0
4,4,1590,tt0052182,4.0
...,...,...,...,...
787536,812812,1032,tt0083530,3.0
787537,812813,99,tt0107798,3.0
787538,812814,333,tt0093857,3.0
787539,812815,49,tt0144168,3.0


In [181]:
omdb.head()

,Title,Year,Rated,Runtime,Writer,Plot,Language,imdbRating,imdbVotes,imdbID,...,Series,Released_season,Released_month,Released_day,PG_Rating,Available_languages,Oscars_won,Oscars_nominated,Golden_globe_won,Golden_globe_nominated
0,Toy Story,1995.0,G,81.0,"John Lasseter (original story by), Pete Docter...",A cowboy doll is profoundly threatened and jea...,['English'],8.3,852896.0,tt0114709,...,0,4.0,11.0,22.0,0.0,1,0.0,3.0,0.0,0.0
1,Jumanji,1995.0,PG,104.0,"Jonathan Hensleigh (screenplay by), Greg Taylo...",When two kids find and play a magical board ga...,"['English', ' French']",7.0,294340.0,tt0113497,...,0,1.0,12.0,15.0,1.0,2,0.0,0.0,0.0,0.0
2,Grumpy Old Men,1993.0,PG-13,103.0,Mark Steven Johnson,A lifelong feud between two neighbors since ch...,['English'],7.0,41401.0,tt0107050,...,0,1.0,12.0,25.0,2.0,1,0.0,0.0,0.0,0.0
3,Waiting to Exhale,1995.0,R,124.0,"Terry McMillan (novel), Terry McMillan (screen...","Based on Terry McMillan's novel, this film fol...",['English'],5.9,9222.0,tt0114885,...,0,1.0,12.0,22.0,3.0,1,0.0,0.0,0.0,0.0
4,Father of the Bride Part II,1995.0,PG,106.0,"Albert Hackett (screenplay ""Father's Little Di...",George Banks must deal not only with the pregn...,['English'],6.0,33005.0,tt0113041,...,0,1.0,12.0,8.0,1.0,1,0.0,0.0,0.0,1.0


In [182]:
omdb.columns

Index(['Title', 'Year', 'Rated', 'Runtime', 'Writer', 'Plot', 'Language',
       'imdbRating', 'imdbVotes', 'imdbID', 'Rotten Tomatoes', 'Metacritic',
       'Series', 'Released_season', 'Released_month', 'Released_day',
       'PG_Rating', 'Available_languages', 'Oscars_won', 'Oscars_nominated',
       'Golden_globe_won', 'Golden_globe_nominated'],
      dtype='object')

In [183]:
# Add movies which could not be retrieved by omdb
imdb_ids = pd.DataFrame(movies['imdbID'].unique()).rename(columns={0: 'imdbID'})
omdb = imdb_ids.merge(omdb, how='left', on='imdbID')

In [241]:
omdb

,imdbID,Title,Year,Rated,Runtime,Writer,Plot,Language,imdbRating,imdbVotes,Rotten Tomatoes,Metacritic,Series,Released_season,Released_month,Released_day,PG_Rating,Available_languages,Awards
0,tt0114709,Toy Story,1995.0,G,81.0,"John Lasseter (original story by), Pete Docter...",A cowboy doll is profoundly threatened and jea...,['English'],8.3,852896.0,10.0,9.5,0.0,4.0,11.0,22.0,0.0,1.0,3.0
1,tt0113497,Jumanji,1995.0,PG,104.0,"Jonathan Hensleigh (screenplay by), Greg Taylo...",When two kids find and play a magical board ga...,"['English', ' French']",7.0,294340.0,5.4,3.9,0.0,1.0,12.0,15.0,1.0,2.0,0.0
2,tt0107050,Grumpy Old Men,1993.0,PG-13,103.0,Mark Steven Johnson,A lifelong feud between two neighbors since ch...,['English'],7.0,41401.0,6.3,5.3,0.0,1.0,12.0,25.0,2.0,1.0,0.0
3,tt0114885,Waiting to Exhale,1995.0,R,124.0,"Terry McMillan (novel), Terry McMillan (screen...","Based on Terry McMillan's novel, this film fol...",['English'],5.9,9222.0,5.6,0.0,0.0,1.0,12.0,22.0,3.0,1.0,0.0
4,tt0113041,Father of the Bride Part II,1995.0,PG,106.0,"Albert Hackett (screenplay ""Father's Little Di...",George Banks must deal not only with the pregn...,['English'],6.0,33005.0,4.8,4.9,0.0,1.0,12.0,8.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9418,tt0953318,Ben X,2007.0,Not Rated,93.0,"Nic Balthazar (novel), Rik D'Hiet (advisor on ...",As an alternative to getting bullied at school...,"['Flemish', ' Dutch']",7.4,18103.0,6.8,0.0,0.0,4.0,9.0,26.0,NaN,2.0,0.0
9419,tt0960731,Bedtime Stories,2008.0,PG,99.0,"Matt Lopez (screenplay), Tim Herlihy (screenpl...",A hotel handyman's life changes when the lavis...,['English'],6.0,86128.0,2.6,3.3,0.0,1.0,12.0,25.0,1.0,1.0,0.0
9420,tt0025464,Manhattan Melodrama,1934.0,NOT RATED,93.0,"Oliver H.P. Garrett (screen play), Joseph L. M...",The friendship between two orphans endures eve...,['English'],7.2,3084.0,8.0,0.0,0.0,2.0,5.0,4.0,NaN,1.0,0.0
9421,tt1024715,Choke,2008.0,R,92.0,"Clark Gregg (screenplay), Chuck Palahniuk (novel)",A sex-addicted con-man pays for his mother's h...,['English'],6.4,31058.0,5.5,4.7,0.0,4.0,9.0,26.0,3.0,1.0,0.0


In [242]:
ratings

,Unnamed: 0,user_id,imdbID,rating
0,0,1264,tt0047034,3.5
1,1,213,tt0304141,2.5
2,2,593,tt0369436,3.0
3,3,609,tt1077258,4.0
4,4,1590,tt0052182,4.0
...,...,...,...,...
787536,812812,1032,tt0083530,3.0
787537,812813,99,tt0107798,3.0
787538,812814,333,tt0093857,3.0
787539,812815,49,tt0144168,3.0


In [294]:
 # Dropping the 10 almost empty movies
indices = omdb[omdb['imdbRating'].isna()]['imdbID'].index
for i in indices:
    omdb = omdb.drop([i], axis=0, )
omdb = omdb.reset_index(drop=True)

In [185]:
# Replace for Series, PG and awards NaN with 0 and handle accordingly
for i in range(9, 14):
    omdb.iloc[:, i] = omdb.iloc[:, i].fillna(0)

In [186]:
 # Fill NaN Runtime
omdb.loc[:, 'Runtime'] = omdb.loc[:, 'Runtime'].fillna(omdb.loc[:, 'Runtime'].median())
# Fill NaN for imdbVotes
omdb.loc[:, 'imdbVotes'] = omdb.loc[:, 'imdbVotes'].fillna(omdb.loc[:, 'imdbVotes'].median())

In [187]:
#Combine awards in one column
omdb['Awards'] = omdb['Oscars_won'] + omdb['Golden_globe_won'] + omdb['Oscars_nominated'] + omdb['Golden_globe_nominated']
omdb = omdb.drop(columns={'Oscars_won', 'Oscars_nominated','Golden_globe_won', 'Golden_globe_nominated'})

In [188]:
# Select relevant columns NOCHMAL PRÜFEN
features = omdb[['Year', 'Runtime', 'imdbVotes', 'Series', 'Awards']]
features

,Year,Runtime,imdbVotes,Series,Awards
0,1995.0,81.0,852896.0,0.0,3.0
1,1995.0,104.0,294340.0,0.0,0.0
2,1993.0,103.0,41401.0,0.0,0.0
3,1995.0,124.0,9222.0,0.0,0.0
4,1995.0,106.0,33005.0,0.0,1.0
...,...,...,...,...,...
9418,2007.0,93.0,18103.0,0.0,0.0
9419,2008.0,99.0,86128.0,0.0,0.0
9420,1934.0,93.0,3084.0,0.0,0.0
9421,2008.0,92.0,31058.0,0.0,0.0


In [281]:
omdb['imdbID'].unique().size

9413

In [286]:
#entferne Ratings von den 10 Movies, für die Feature fehlen
ratings=ratings.merge(omdb[['imdbID', 'Runtime']], on='imdbID', how='left').dropna()

## KNN

In [189]:
# Standardize the features
scaler = preprocessing.StandardScaler()
features = scaler.fit_transform(features)

In [190]:
features

array([[ 0.37902598, -0.99101573,  6.82182467, -0.08076304,  2.66011286],
       [ 0.37902598, -0.04136799,  2.10292706, -0.08076304, -0.39288153],
       [ 0.27127413, -0.08265702, -0.03399966, -0.08076304, -0.39288153],
       ...,
       [-2.90740551, -0.49554734, -0.35771653, -0.08076304, -0.39288153],
       [ 1.07941302, -0.53683638, -0.12138133, -0.08076304, -0.39288153],
       [ 0.59452969, -2.97288929, -0.26963364, -0.08076304, -0.39288153]])

In [191]:
# features of the first entry (imdbID 439)
features[0]

array([ 0.37902598, -0.99101573,  6.82182467, -0.08076304,  2.66011286])

In [192]:
features.shape[0]

9413

In [193]:
# create NearesNeighbor model with the minkowski distance
knn = NearestNeighbors(metric='minkowski', p=2, algorithm='brute', n_jobs=-1)

In [194]:
# fit to all the features
knn.fit(features)

NearestNeighbors(algorithm='brute', n_jobs=-1)

In [195]:
# create two array for distance and index to store
distances = np.empty((features.shape[0],features.shape[0]))
indices = np.empty((features.shape[0],features.shape[0]))

# iterate over all imdbIDs
for i in range(0, features.shape[0]):
    # get all neighbors and the distances for each imdbID i
    distances[i], indices[i] = knn.kneighbors([features[i]], n_neighbors=features.shape[0])
    
    if i%1000==0:
        print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [196]:
distances

array([[0.00000000e+00, 9.76860350e-01, 1.13566536e+00, ...,
        2.14837833e+01, 2.50220733e+01, 2.62568299e+01],
       [0.00000000e+00, 2.31530461e-01, 3.41213480e-01, ...,
        1.92357094e+01, 2.37910878e+01, 2.43821886e+01],
       [0.00000000e+00, 4.47059814e-02, 4.92324914e-02, ...,
        1.99438068e+01, 2.39671763e+01, 2.43005468e+01],
       ...,
       [0.00000000e+00, 1.77078205e-02, 8.51182829e-02, ...,
        2.05383120e+01, 2.46167814e+01, 2.49337776e+01],
       [0.00000000e+00, 4.58409513e-02, 8.47872230e-02, ...,
        2.00808711e+01, 2.43612376e+01, 2.46843064e+01],
       [4.21468485e-08, 3.27703914e-01, 3.28203217e-01, ...,
        2.20200229e+01, 2.64780039e+01, 2.68148825e+01]])

In [197]:
indices

array([[0.000e+00, 1.495e+03, 3.510e+02, ..., 6.204e+03, 3.792e+03,
        6.155e+03],
       [1.000e+00, 6.800e+01, 1.263e+03, ..., 6.204e+03, 3.792e+03,
        6.155e+03],
       [2.000e+00, 5.240e+02, 4.990e+02, ..., 3.060e+02, 3.792e+03,
        6.155e+03],
       ...,
       [9.410e+03, 8.646e+03, 8.062e+03, ..., 3.060e+02, 3.792e+03,
        6.155e+03],
       [9.411e+03, 9.089e+03, 9.347e+03, ..., 3.060e+02, 3.792e+03,
        6.155e+03],
       [9.412e+03, 7.817e+03, 4.377e+03, ..., 6.204e+03, 3.792e+03,
        6.155e+03]])

In [198]:
len(indices)

9413

In [199]:
distances_df = pd.DataFrame(distances)
distances_df

,0,1,2,3,4,5,6,7,8,9,...,9403,9404,9405,9406,9407,9408,9409,9410,9411,9412
0,0.000000e+00,0.976860,1.135665,1.699001,1.890985,1.910380,2.099037,2.133407,2.187329,2.283324,...,16.622917,16.751525,17.055238,18.268484,18.615190,19.040123,19.225665,21.483783,25.022073,26.256830
1,0.000000e+00,0.231530,0.341213,0.341806,0.367083,0.407168,0.440377,0.482189,0.494031,0.553896,...,14.863361,16.483559,16.518907,17.070838,17.129170,17.280192,17.968719,19.235709,23.791088,24.382189
2,0.000000e+00,0.044706,0.049232,0.066075,0.067919,0.069631,0.097948,0.104702,0.114911,0.118720,...,15.541984,16.395306,16.947454,16.994055,17.150865,18.599239,19.123745,19.943807,23.967176,24.300547
3,0.000000e+00,0.041506,0.058528,0.067969,0.069797,0.071733,0.076336,0.081456,0.085922,0.092712,...,15.726031,15.846464,16.423855,16.432732,16.577583,18.257705,18.790115,20.146446,23.268899,23.556515
4,0.000000e+00,0.054051,0.098913,0.109120,0.125263,0.140933,0.141384,0.153273,0.158954,0.161374,...,15.224892,16.346573,16.590510,16.946122,16.977143,18.568410,19.028987,19.660782,23.846996,24.212594
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9408,0.000000e+00,0.004342,0.009319,0.024644,0.030220,0.031124,0.042033,0.047066,0.061832,0.062249,...,15.785130,16.681266,17.193539,17.329620,17.457492,18.825210,19.565828,20.176415,24.337793,24.647604
9409,0.000000e+00,0.058353,0.079417,0.086282,0.108915,0.109551,0.122021,0.124029,0.124337,0.136283,...,15.234460,16.526583,17.048975,17.174405,17.300138,18.227520,19.332984,19.621367,24.067904,24.443593
9410,0.000000e+00,0.017708,0.085118,0.099590,0.099618,0.109598,0.168509,0.188383,0.203639,0.231117,...,16.211080,16.978427,17.409454,17.590399,17.681161,19.365054,19.722099,20.538312,24.616781,24.933778
9411,0.000000e+00,0.045841,0.084787,0.085873,0.087504,0.093174,0.099119,0.102210,0.104329,0.107912,...,15.693239,16.710679,17.219971,17.363454,17.489265,18.722679,19.611146,20.080871,24.361238,24.684306


In [200]:
indices_df = pd.DataFrame(indices)
indices_df

,0,1,2,3,4,5,6,7,8,9,...,9403,9404,9405,9406,9407,9408,9409,9410,9411,9412
0,0.0,1495.0,351.0,49.0,1079.0,460.0,8967.0,1513.0,4399.0,9133.0,...,9333.0,6634.0,6768.0,8313.0,5551.0,7731.0,7687.0,6204.0,3792.0,6155.0
1,1.0,68.0,1263.0,205.0,2294.0,5319.0,220.0,4962.0,3924.0,8218.0,...,6768.0,9092.0,5551.0,8313.0,7687.0,7731.0,306.0,6204.0,3792.0,6155.0
2,2.0,524.0,499.0,526.0,2617.0,2923.0,511.0,532.0,279.0,348.0,...,343.0,5551.0,8313.0,7687.0,7731.0,9092.0,6204.0,306.0,3792.0,6155.0
3,3.0,25.0,780.0,961.0,347.0,951.0,905.0,619.0,134.0,7956.0,...,343.0,5551.0,7687.0,8313.0,7731.0,6204.0,9092.0,306.0,3792.0,6155.0
4,4.0,329.0,56.0,894.0,579.0,472.0,332.0,761.0,1222.0,1656.0,...,343.0,5551.0,8313.0,7687.0,7731.0,9092.0,6204.0,306.0,3792.0,6155.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9408,9408.0,8888.0,9140.0,8901.0,8961.0,8877.0,8795.0,8723.0,9125.0,8677.0,...,343.0,5551.0,8313.0,7687.0,7731.0,9092.0,6204.0,306.0,3792.0,6155.0
9409,9409.0,9291.0,9066.0,9269.0,3762.0,9246.0,8902.0,9065.0,8968.0,8928.0,...,343.0,5551.0,8313.0,7687.0,7731.0,9092.0,6204.0,306.0,3792.0,6155.0
9410,9410.0,8646.0,8062.0,8088.0,1334.0,6998.0,5435.0,7537.0,8542.0,6487.0,...,343.0,5551.0,7687.0,8313.0,7731.0,9092.0,6204.0,306.0,3792.0,6155.0
9411,9411.0,9089.0,9347.0,8865.0,9354.0,7196.0,9206.0,286.0,8961.0,8643.0,...,343.0,5551.0,8313.0,7687.0,7731.0,9092.0,6204.0,306.0,3792.0,6155.0


In [239]:
#Alles in eine Funktion
def predict_rating(imdbID, userID):
    
    #Bestimme Index in omdb von der ImdbID
    index = omdb.loc[(omdb['imdbID'] == imdbID)].index

    #Speichere Reihenfolge der Indizes der Nachbarn des Films und tausche erste Reihe mit Spaltenbezeichnungen
    movie_row = indices_df.loc[index]
    movie_row = movie_row.reset_index(drop=True)
    movie_row.columns = movie_row.loc[0]
    movie_row.drop(0, inplace=True)
    movie_row.loc[1,:] = range(0,len(indices_df))
    
    #Speichere alle Filme, die der User bewertet hat und bestimme index davon
    rated_id = ratings.loc[ratings['user_id'] == userID]
    rated = np.empty((len(rated_id), 1))
    i = 0
    for imdb in rated_id['imdbID']:
        rated[i] = omdb.loc[(omdb['imdbID'] == imdb)].index
        i = i+1
    rated_df = pd.DataFrame(rated)
    
    # Bestimme die Position der bewerteten Filme innerhalb der Nachbarschaft
    position = movie_row[list(rated_df[0].astype(int))]
    position = position.transpose()
    position.columns = ['position']
    position ['Index'] = position.index
    position = position.reset_index(drop = True)
        
    #Greife auf den Film in der Distanzmatrix zu
    movie_row_distance = distances_df.loc[index]
    
    #Bestimme Distanz der Filme vom Film, die der User bewertet hat
    user_distances = movie_row_distance[list(position['position'].astype(int))].transpose()
    user_distances.columns = ['distance']
    user_distances =user_distances.reset_index(drop = True)
    
    #Füge alle Infos zusammen und sortiere von nächstem zum weitest entfernten Film
    neighbors = pd.concat([position, user_distances], axis=1, join = "inner")
    neighbors = neighbors.sort_values("distance")
    
    #Füge die Ratings des Users zu diesen Filmen noch hinzu
    rated_id['Index']= rated
    neighbors = neighbors.merge(rated_id[["rating","Index"]], on="Index", how = "outer")
    
    # Berechne die Prognose: Gewichtung der Bewertung der Nachbarn je nach Distanz
    pred = sum(neighbors['rating']*(1+(1-neighbors['distance']/neighbors['distance'].mean()))) / neighbors.shape[0]
    
    return pred

In [236]:
predict_rating('tt0102467', 123)

<ipython-input-235-12c1fc05a7de>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rated_id['Index']= rated


3.588783208835612

In [ ]:
omdb

In [ ]:
predict_rating('tt0114709', 1228)

In [ ]:
n=100
test_set = ratings.sample(n)
test_set   

In [ ]:
predictions = np.empty(n)
reals = np.empty(n)
i = 0

for index, row in test_set.iterrows():
        imdbID = row['imdbID']
        userID = row['user_id']
        real = row['rating']
        pred = predict_rating(imdbID, userID)
        predictions[i] = pred
        reals[i] = real
        i = i+1
rmse = mean_squared_error(reals, predictions, squared=False)
print('RMSE: '+str(rmse))

,imdbID,Title,Year,Rated,Runtime,Writer,Plot,Language,imdbRating,imdbVotes,Rotten Tomatoes,Metacritic,Series,Released_season,Released_month,Released_day,PG_Rating,Available_languages,Awards
0,tt0114709,Toy Story,1995.0,G,81.0,"John Lasseter (original story by), Pete Docter...",A cowboy doll is profoundly threatened and jea...,['English'],8.3,852896.0,10.0,9.5,0.0,4.0,11.0,22.0,0.0,1.0,3.0
1,tt0113497,Jumanji,1995.0,PG,104.0,"Jonathan Hensleigh (screenplay by), Greg Taylo...",When two kids find and play a magical board ga...,"['English', ' French']",7.0,294340.0,5.4,3.9,0.0,1.0,12.0,15.0,1.0,2.0,0.0
2,tt0107050,Grumpy Old Men,1993.0,PG-13,103.0,Mark Steven Johnson,A lifelong feud between two neighbors since ch...,['English'],7.0,41401.0,6.3,5.3,0.0,1.0,12.0,25.0,2.0,1.0,0.0
3,tt0114885,Waiting to Exhale,1995.0,R,124.0,"Terry McMillan (novel), Terry McMillan (screen...","Based on Terry McMillan's novel, this film fol...",['English'],5.9,9222.0,5.6,0.0,0.0,1.0,12.0,22.0,3.0,1.0,0.0
4,tt0113041,Father of the Bride Part II,1995.0,PG,106.0,"Albert Hackett (screenplay ""Father's Little Di...",George Banks must deal not only with the pregn...,['English'],6.0,33005.0,4.8,4.9,0.0,1.0,12.0,8.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9418,tt0953318,Ben X,2007.0,Not Rated,93.0,"Nic Balthazar (novel), Rik D'Hiet (advisor on ...",As an alternative to getting bullied at school...,"['Flemish', ' Dutch']",7.4,18103.0,6.8,0.0,0.0,4.0,9.0,26.0,NaN,2.0,0.0
9419,tt0960731,Bedtime Stories,2008.0,PG,99.0,"Matt Lopez (screenplay), Tim Herlihy (screenpl...",A hotel handyman's life changes when the lavis...,['English'],6.0,86128.0,2.6,3.3,0.0,1.0,12.0,25.0,1.0,1.0,0.0
9420,tt0025464,Manhattan Melodrama,1934.0,NOT RATED,93.0,"Oliver H.P. Garrett (screen play), Joseph L. M...",The friendship between two orphans endures eve...,['English'],7.2,3084.0,8.0,0.0,0.0,2.0,5.0,4.0,NaN,1.0,0.0
9421,tt1024715,Choke,2008.0,R,92.0,"Clark Gregg (screenplay), Chuck Palahniuk (novel)",A sex-addicted con-man pays for his mother's h...,['English'],6.4,31058.0,5.5,4.7,0.0,4.0,9.0,26.0,3.0,1.0,0.0


In [295]:
predict_rating('tt0114709', 1228)

<ipython-input-239-12c1fc05a7de>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rated_id['Index']= rated


2.9942627104030706

In [298]:
n=100
test_set = ratings.sample(n)
test_set   

,Unnamed: 0,user_id,imdbID,rating,Runtime
86907,89269,1220,tt0099810,3.0,135.0
340117,349962,144,tt0054047,4.0,128.0
726463,749400,1443,tt0109040,4.0,86.0
676725,697818,557,tt0169547,5.0,122.0
588317,606323,478,tt0467200,2.0,115.0
...,...,...,...,...,...
628924,648290,1061,tt0266915,4.0,90.0
284550,292697,94,tt0386117,3.0,101.0
320126,329365,271,tt0083866,4.0,115.0
673384,694353,268,tt0109424,3.5,102.0


In [307]:
predictions = np.empty(n)
reals = np.empty(n)
i = 0

for index, row in test_set.iterrows():
        imdbID = row['imdbID']
        userID = row['user_id']
        real = row['rating']
        pred = predict_rating(imdbID, userID)
        predictions[i] = pred
        reals[i] = real
        i = i+1
rmse = mean_squared_error(reals, predictions, squared=False)
print('RMSE: '+str(rmse))

<ipython-input-239-12c1fc05a7de>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rated_id['Index']= rated


RMSE: 0.8746572902228813
